In [12]:
from pyannote.audio.tasks import __all__ as TASKS; print('\n'.join(TASKS))

SpeakerDiarization
VoiceActivityDetection
OverlappedSpeechDetection
MultiLabelSegmentation
SpeakerEmbedding
Segmentation


In [1]:
from pyannote.database import registry
import os
registry.load_database(
    os.path.join('/mnt/', 'e', 'Files', 'Acoustic_Data', 'Datasets_specified', 'yaml','My_Databases_specified.yml')
    )
os.environ["PYANNOTE_DATABASE_CONFIG"] = os.path.join('/mnt/', 'e', 'Files', 'Acoustic_Data', 'Datasets_specified','yaml', 'My_Databases_specified.yml')

'My_datasets_specified.SpeakerDiarization.Detection' found in /mnt/e/Files/Acoustic_Data/Datasets_specified/yaml/My_Databases_specified.yml does not define the 'scope' of speaker labels (file, database, or global). Setting it to 'file'.


In [ ]:
from pyannote.database import get_protocol, FileFinder
preprocessors = {"audio": FileFinder()}
cow_audio = get_protocol('My_datasets_specified.SpeakerDiarization.Detection', 
                   preprocessors=preprocessors)

In [2]:
from pyannote.audio.tasks import MultiLabelSegmentation
task = MultiLabelSegmentation(protocol=cow_audio, 
    duration=1.0, 
    batch_size=32,
    classes= ["rumination","hoofbeat","breath","gulp","grazing"]
    )

/home/wuxiaohui9707/miniconda3/envs/myenv/lib/python3.8/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/home/wuxiaohui9707/miniconda3/envs/myenv/lib/python3.8/site-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


Protocol My_datasets_specified.SpeakerDiarization.Detection does not precompute the output of torchaudio.info(): adding a 'torchaudio.info' preprocessor for you to speed up dataloaders. See pyannote.database documentation on how to do that yourself.


In [3]:
from pyannote.audio.models import custom_model
model = custom_model.SoundEventDetection(task=task)

#from pyannote.audio.models.segmentation import PyanNet
#model = PyanNet(sincnet={'stride': 10}, task=task)

model.to("cuda")

MySoundEventDetection(
  (mfcc): MFCC(
    (amplitude_to_DB): AmplitudeToDB()
    (MelSpectrogram): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (linear1): Linear(in_features=40, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=16, bias=True)
)

In [4]:
import pytorch_lightning as pl
output_directory = os.path.join('/mnt/', 'e', 'Files', 'Acoustic_Data', 'Datasets_specified')
trainer = pl.Trainer(devices=1, accelerator="gpu", max_epochs=10, default_root_dir=output_directory)
trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type             | Params | In sizes      | Out sizes     
----------------------------------------------------------------------------------------
0 | mfcc              | MFCC             | 0      | [1, 1, 16000] | [1, 1, 40, 81]
1 | linear1           | Linear           | 1.3 K  | [1, 81, 40]   | [1, 81, 32]   
2 | linear2           | Linear           | 528    | [1, 81, 32]   | [1, 81, 16]   
3 | classifier      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [11]:
from pyannote.audio.pipelines import MultiLabelSegmentation
test_file = next(cow_audio.test())
pipeline = MultiLabelSegmentation(segmentation=model)
initial_params = {
    "thresholds": {
        "rumination": {"onset": 0.8, "offset": 0.6, "min_duration_on": 0.01, "min_duration_off": 0.00},
        "breath": {"onset": 0.8, "offset": 0.6, "min_duration_on": 0.01, "min_duration_off": 0.01},
        "hoofbeat": {"onset": 0.8, "offset": 0.6, "min_duration_on": 0.005, "min_duration_off": 0.005},
        "grazing": {"onset": 0.8, "offset": 0.6, "min_duration_on": 0.005, "min_duration_off": 0.005},
        "gulp": {"onset": 0.8, "offset": 0.6, "min_duration_on": 0.005, "min_duration_off": 0.005},
    }
}
pipeline.instantiate(initial_params)
detection = pipeline(test_file)
print(detection)

[ 00:00:00.006 -->  00:00:00.030] A grazing
[ 00:00:00.055 -->  00:00:00.228] B grazing
[ 00:00:00.240 -->  00:00:00.759] C grazing
[ 00:00:00.771 -->  00:00:00.845] D grazing
[ 00:00:00.858 -->  00:00:01.043] E grazing
[ 00:00:01.055 -->  00:00:01.067] F grazing
[ 00:00:01.080 -->  00:00:01.191] G grazing
[ 00:00:01.277 -->  00:00:01.635] H grazing
[ 00:00:01.660 -->  00:00:01.771] I grazing
[ 00:00:01.820 -->  00:00:02.450] J grazing
[ 00:00:02.500 -->  00:00:04.611] K grazing
[ 00:00:04.623 -->  00:00:04.858] L grazing
[ 00:00:04.895 -->  00:00:04.956] M grazing
[ 00:00:04.969 -->  00:00:05.006] N grazing
[ 00:00:05.006 -->  00:00:05.018] A gulp
[ 00:00:05.030 -->  00:00:05.524] O grazing
[ 00:00:05.561 -->  00:00:05.672] P grazing
[ 00:00:05.685 -->  00:00:05.746] Q grazing
[ 00:00:05.771 -->  00:00:05.796] R grazing
[ 00:00:05.845 -->  00:00:05.858] S grazing
[ 00:00:05.882 -->  00:00:05.907] T grazing
[ 00:00:05.919 -->  00:00:05.932] U grazing
[ 00:00:05.944 -->  00:00:06.401] V